# Import software libraries

In [1]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [4]:
# Connect to SQLite database.
conn = sqlite3.connect('data/prod_sample.db')
conn
cursor = conn.cursor()

In [5]:
# List all the tables in the database.
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
table_names = [table[0] for table in tables]
print("Tables in the database:")
print(table_names)


Tables in the database:
['stock_description', 'online_retail_history']


# Read data from the `online_retail_history` table

In [7]:
# Write the query to be executed that selects everything from the online_retail_history table.
query_history = 'SELECT * FROM online_retail_history'



# Use the read_sql function in pandas to read a query into a DataFrame.
history = pd.read_sql(query_history, conn)


# Preview the first five rows of the data.
history.head(n=5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [9]:
# Get the shape of the data.
history.shape

(15321, 8)

# Read data from the `stock_description` table

In [10]:
# Write the query to be executed that selects everything from the online_retail_history table.
query_stock = 'SELECT * FROM stock_description'



# Use the read_sql function in pandas to read a query into a DataFrame.
stock = pd.read_sql(query_stock, conn)


# Preview the first five rows of the data.
stock.head(n=5)

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [11]:
# Get the shape of the data.
stock.shape


(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [12]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.
query = '''
SELECT 
    o.StockCode, 
    d.Description,
    o.* 
FROM online_retail_history o
JOIN stock_description d ON o.StockCode = d.StockCode;
'''





# Use the read_sql function in pandas to read a query into a DataFrame.
agg_table = pd.read_sql(query, conn)


# Preview the first five rows of the data.
agg_table.head(n=5)


,StockCode,Description,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,84879,ASSORTED COLOUR BIRD ORNAMENT,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,20725,LUNCH BAG RED RETROSPOT,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [13]:
# Get the shape of the data.
agg_table.shape

(17032, 10)

# Identify and fix corrupt or unusable data

In [16]:
# Check the value counts of the "Description" field.
description_value_counts = agg_table['Description'].value_counts()
print(description_value_counts)


CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
REGENCY CAKESTAND 3 TIER              1711
?                                     1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64


In [17]:
# Remove rows where "Description" is just a question mark (?).
cleaned_data_1 = agg_table[agg_table['Description'] != '?']






# Preview the first five rows of the data.
cleaned_data_1.head(n=5)


,StockCode,Description,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,84879,ASSORTED COLOUR BIRD ORNAMENT,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,20725,LUNCH BAG RED RETROSPOT,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


# Identify and remove duplicates

In [22]:
# Identify all duplicated data.
duplicated_data = \
cleaned_data_1[cleaned_data_1.duplicated(keep=False)]
print('NUmber of rows with duplicated data:', duplicated_data.shape[0])






NUmber of rows with duplicated data: 223


In [25]:
# Print the duplicated data.
duplicated_data.head(n=5)


,StockCode,Description,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
176,20727,LUNCH BAG BLACK SKULL.,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65
178,20727,LUNCH BAG BLACK SKULL.,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65
496,84879,ASSORTED COLOUR BIRD ORNAMENT,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52
497,84879,ASSORTED COLOUR BIRD ORNAMENT,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52
569,20725,LUNCH BAG RED RETROSPOT,537955,20725,1,2010-12-09 11:28:00,1.65,u16782,United Kingdom,1.65


In [24]:
# Remove the duplicated data.
cleaned_data_final = \
cleaned_data_1[~cleaned_data_1.duplicated()]






# Preview the first five rows of the data.
cleaned_data_final.head(n=5)

,StockCode,Description,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,84879,ASSORTED COLOUR BIRD ORNAMENT,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,20725,LUNCH BAG RED RETROSPOT,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


# Correct date formats

In [26]:
# Get the data types for every column in the DataFrame.
cleaned_data_final.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StockCode    15206 non-null  object 
 1   Description  15206 non-null  object 
 2   Invoice      15206 non-null  object 
 3   StockCode    15206 non-null  object 
 4   Quantity     15206 non-null  int64  
 5   InvoiceDate  15206 non-null  object 
 6   Price        15194 non-null  float64
 7   CustomerID   12435 non-null  object 
 8   Country      15206 non-null  object 
 9   TotalAmount  15194 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 1.3+ MB


In [27]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.
cleaned_data_final['InvoiceDate'] = \
pd.to_datetime(cleaned_data_final['InvoiceDate'],
              format = "%Y-%m-%d" )



In [28]:
# Get the data types for every column in the converted DataFrame.
cleaned_data_final.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   StockCode    15206 non-null  object        
 1   Description  15206 non-null  object        
 2   Invoice      15206 non-null  object        
 3   StockCode    15206 non-null  object        
 4   Quantity     15206 non-null  int64         
 5   InvoiceDate  15206 non-null  datetime64[ns]
 6   Price        15194 non-null  float64       
 7   CustomerID   12435 non-null  object        
 8   Country      15206 non-null  object        
 9   TotalAmount  15194 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 1.3+ MB


# Examine the table before finishing

In [29]:
# Preview the first five rows of the data.
cleaned_data_final.head(n=5)


,StockCode,Description,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,84879,ASSORTED COLOUR BIRD ORNAMENT,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,85123A,CREAM HANGING HEART T-LIGHT HOLDER,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,20725,LUNCH BAG RED RETROSPOT,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


# Load the dataset into a pickle file

In [30]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
cleaned_data_final.to_pickle('online_history_cleaned.pickle')


In [31]:
# Close any connections to the database.

conn.close()